In [1]:
import pandas as pd

In [2]:
print('", "'.join(("id	c	part	vp1	up1	vp2	up2	desc	sup	mfg	pn	proj	sec	dwg	dwg_id	qty	spare	po	resp	ord	wk	s	each	d	n	l	b	w	upd	lc").split('	')))

id", "c", "part", "vp1", "up1", "vp2", "up2", "desc", "sup", "mfg", "pn", "proj", "sec", "dwg", "dwg_id", "qty", "spare", "po", "resp", "ord", "wk", "s", "each", "d", "n", "l", "b", "w", "upd", "lc


In [3]:
# BOM for a specific project
# sheet_name = the name of the sheet containing the BOM
# header = index for the row containing the headers 
bom_path = "G:\\Shared drives\\General\\QTC\\Manufacturing\\P654 - St Jude Childrens\\02. Project Docs\\P654 - Project Doc St Jude AM 11 04 22.xlsx"
project_number = bom_path.split("\\")[-1][1:4]
project_bom = pd.read_excel(bom_path, 
                            sheet_name="BOM", 
                            header=6)

# Remove unneeded columns
project_bom.drop(['Tot (CAD)'], axis=1, inplace=True)
# Make all column headers lower case for consistency
project_bom.rename({name: name.lower() for name in project_bom.columns}, axis=1, inplace=True)
# Remove completely empty rows
project_bom.dropna(how='all', inplace=True)
# Remove rows with a section tag 
project_bom = project_bom[project_bom['section'].isnull()]

# Reset index after removing all unrelevant rows
project_bom.reset_index()

print(project_bom.iloc[0])


section                        NaN
item description         Coldhead 
supplier                    Pride 
manufacturer                   NaN
p/n                       KDE415SA
sec                            NaN
dwg                            NaN
identifier                   CH501
qty                            1.0
lead time (wks)             CH501 
est deliv.                     NaN
responsible         Paras / David 
location                       NaN
po                           85429
stat                           2.0
old po \nref                   NaN
each (cad)                     NaN
usd                        20950.0
budget (cad)                   NaN
unnamed: 20                  86291
unnamed: 21                    NaN
unnamed: 22                    NaN
Name: 3, dtype: object


In [6]:
db_path = "G:\\Shared drives\\General\\QTC\\Manufacturing\\P698 - Database\\BackUps\\testversion_p698_database.xlsx"

# Create output dataframe
db_headers = ["id", "c", "part", "vp1", "up1", "vp2", "up2", "desc", "sup", "mfg", "pn", "proj", "sec", "dwg", "dwg_id", "qty", "spare", "po", "resp", "ord", "wk", "s", "each", "d", "n", "l", "b", "w", "upd", "lc"]
output_df = pd.DataFrame(columns=db_headers)

# Map columns to the ouput data frame
output_df['desc'] = project_bom['item description']
output_df['sup'] = project_bom['supplier']
output_df['mfg'] = project_bom['manufacturer ']
output_df['pn'] = project_bom['p/n']
output_df['proj'] = project_number
output_df['dwg_id'] = project_bom['identifier']
output_df['qty'] = project_bom['qty']
output_df['po'] = project_bom['po']
output_df['s'] = project_bom['stat']
output_df['resp'] = project_bom['responsible']
output_df['each'] = project_bom['each (cad)'].fillna(project_bom['usd'])
output_df['d'] = ['U' if pd.isna(x) else 'C' for x in project_bom['each (cad)']]

# Reset the index of the dataframe 
output_df.reset_index(drop=True, inplace=True)

# Populate some columns based on part numbers
cols = ['part', 'vp1', 'up1', 'vp2', 'up2']

# Concatenate all current sheets into a 'mega' database 
main_db = pd.read_excel(db_path, None)
main_db.pop("400-500's")
main_db = pd.concat(main_db.values())

# Grab part numbers and populate output dataframe based on other sheets
for idx, pn in enumerate(output_df['pn']):
    if not pd.isna(pn):
        # Find another example of the same part 
        try:
            old_part = main_db.loc[main_db['pn'] == pn].iloc[0]
            for col in cols:
                output_df.at[idx, col] = old_part[col]
        except:
            print(f'part at {idx} could not be populated: {pn}')
            pass



# Write dataframe to xlsx
with pd.ExcelWriter(db_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    output_df.to_excel(writer, sheet_name=str(project_number), index=False)

part at 2 could not be populated: CMSH-250-7.63N
part at 12 could not be populated: VARLREL1/2-10
part at 34 could not be populated: QLHe20-ATSS-F-301
part at 39 could not be populated: 0437516HN
part at 45 could not be populated: N231-0125BR
part at 50 could not be populated: SFC-2-2N
part at 51 could not be populated: SME-2-2N
part at 56 could not be populated: KSV25F-240
part at 57 could not be populated: BUFBRBN8.00SS
part at 63 could not be populated: WQ903
part at 68 could not be populated: SHCV1-S4-0.3P-SS
part at 70 could not be populated: SBV210-S8-SS
part at 71 could not be populated: SBVA210D-S8-SS
part at 72 could not be populated: SBV210-S4-SS
part at 73 could not be populated: SC8256A002V-24VDC
part at 74 could not be populated: SC8210G094-24VDC
part at 75 could not be populated: QF25-050-SBB
part at 85 could not be populated: NMC-250-025
part at 87 could not be populated: N268-050BR
part at 88 could not be populated: 48-4B
part at 89 could not be populated: N64-84
part a